In [1]:
# * IMPORTS
import os
import threading
import time
from datetime import datetime, time
from ipywidgets import widgets
from functools import partial
from config import DataConfig, StyleConfig
from IPython.display import display, clear_output, HTML
from visuals.map_manager import createBaseMap
from visuals.gui_manager import create_stations_cbox, reset_cbox
from app.route import process_route


In [2]:
# we check if we run in a notebook or a voila app
# we will not display the map by default if in a voila app
running_in_voila = os.environ.get('SERVER_SOFTWARE','jupyter').startswith('voila')


In [3]:
# * LOGIC
initial_map = createBaseMap(zoom_start=13)

In [4]:
# * WIDGETS

# header image
from typing import Dict


with open("./images/hero_cropped.jpg", 'rb') as file:
        image = file.read()

header_img = widgets.Image(
    value=image,
    format='jpg',
)
header_img.layout.height = '200px'

# header text
header = widgets.HTML(
    value="<h1>Metro Lyon</h1>",
)

# combo boxes for station selection
cbox_start = create_stations_cbox(placeholder='Choose a Start Point', description='Start')
cbox_end = create_stations_cbox(placeholder='Choose an End Point', description='End')

# buttons to reset the combo-boxes
btn_reset_start = widgets.Button(description="Selection Reset", icon="trash")
btn_reset_end = widgets.Button(description="Selection Reset", icon="trash")

# search button
btn_search = widgets.Button(
    description = "Search",
    disabled = True,
    tooltip = "Search the route between the stations",
    icon = "search"
)
# map button (only shown in Voila app)
btn_map = widgets.Button(
    description = "Show Map",
    disabled = False,
    tooltip = "Show the metro map",
    icon = "map"
)
# progress_bar
progress_bar = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    description='Generating map:',
    style={'bar_color': 'pink'},
    orientation='horizontal'
)

# Hour picker
time_picker = widgets.TimePicker(
    description='Pick a Time',
    disabled=False,
    min=time(0,0,0),
    max=time(23,59,59),
    value=datetime.now().time(),
)
# ! on click only admits a name of the function and it automaticaly puts the button, if we want to pass the cbox, we need partial
btn_reset_start.on_click(partial(reset_cbox, cbox=cbox_start))
btn_reset_end.on_click(partial(reset_cbox, cbox=cbox_end))


def check_search_condition(change):
    if cbox_start.value != "" and cbox_end.value != "":
        btn_search.disabled = False
    else:
        btn_search.disabled = True

# we set what to do when the checkbox changes
cbox_start.observe(check_search_condition, names='value')
cbox_end.observe(check_search_condition, names='value')

# widget that shows the generated route as a list
route_list = widgets.SelectMultiple(
                  options=[],
                  disabled=False
                )
route_list.layout.width = '436px'

# label with the estimated time
lbl_estimated_time = widgets.Label(value="Estimated time of arrival: ")
lbl_travel_time = widgets.Label()


def update_progress(change):
    if change['type'] == 'change' and change['name'] == 'value' and change['new'] == 1:
        threading.Thread(target=process_route).start()

progress_bar.observe(update_progress, 'value')

# output for the progress bar so that we can hide it
out_progress = widgets.Output()

# output for the map, so we can change it
out_map = widgets.Output()

out_btn_map = widgets.Output()


In [5]:
# * PLACEMENT STRUCTURES

hbox_image = widgets.HBox([widgets.Label(), header_img, widgets.Label()], layout=widgets.Layout(justify_content='center'))
hbox_header = widgets.HBox([widgets.Label(), header, widgets.Label()], layout=widgets.Layout(justify_content='center'))

# placement structure for the estimated time and route list
vbox_route = widgets.VBox([lbl_estimated_time, lbl_travel_time, widgets.Label(value="Route:"), route_list ])
# we set the size so that the main tabs window does not change size on changing tab
# vbox_route.layout.height = '160px'
vbox_route.layout.width = '457px'

# placement of the comboboxes with the buttons
vbox_left = widgets.VBox([cbox_start, cbox_end])
vbox_right = widgets.VBox([btn_reset_start, btn_reset_end])
hbox_main = widgets.HBox([vbox_left, vbox_right])

# placement of the search button
hbox_search = widgets.HBox([widgets.Label(), btn_search, widgets.Label()], layout=widgets.Layout(justify_content='center'))
btn_map.layout.width = "100%"

# placement of the map button
hbox_map = widgets.HBox([btn_map], layout=widgets.Layout(justify_content='center'))
# hbox_out_btn_map = widgets.HBox([out_btn_map], layout=widgets.Layout(justify_content='center'))

# elements of the journey tab
journey_tab = widgets.VBox([hbox_main, time_picker, hbox_search, out_progress])

# the tabs widget that starts with only one tab
tabs = widgets.Tab(children=[journey_tab])
tabs.set_title(0, 'Journey Planner')
tabs.layout.justify_content = 'center'

# placement of the tabs widget in the middle of the page
hbox_tabs = widgets.HBox([widgets.Label(), tabs, widgets.Label()], layout=widgets.Layout(justify_content='center'))

In [6]:
# * MAIN DISPLAY

display(HTML(StyleConfig.MATERIAL_CSS.value))
display(hbox_image)
display(hbox_header)


if not running_in_voila:
    with out_map:
        clear_output()
        display(initial_map)
else:
    journey_tab = widgets.VBox([hbox_main, time_picker, hbox_search, hbox_map, out_progress])
    tabs.children = [journey_tab]

display(hbox_tabs)
display(out_map)


# Function to process the route
def on_button_click_search(b):

    origin = cbox_start.value
    target = cbox_end.value

    # we check if we have both stations
    if target != '' and origin != '':
        if target in DataConfig.JUNCTIONS.value.keys():
            target = DataConfig.JUNCTIONS.value[target][0]
        if origin in DataConfig.JUNCTIONS.value.keys():
            origin = DataConfig.JUNCTIONS.value[origin][0]
        # we show the progress bar
        with out_progress:
            clear_output(wait=True)
            display(progress_bar)
            # we run the a_star algorithm and we create the route map
            processed_route = process_route(origin,target, time_picker.value, progress_bar)
            route_list.options = processed_route["user_time_info"]
        # when we are finished    
        if progress_bar.value == 100:
            # we hide the progress bar
            with out_progress:
                clear_output()
            # we change the map with the route map
            with out_map:
                clear_output(wait=True)
                display(processed_route["route_map"])
            # we add a new tab with the route
            lbl_estimated_time.value = f'Estimated time of arrival: {processed_route["arrival_time"]}'
            lbl_travel_time.value = f'Journey time: {processed_route["total_time"]} mins of which {processed_route["travel_time"]} mins of travel and {processed_route["wait_time"]} mins of waiting'
            tabs.children = [journey_tab, vbox_route]
            tabs.set_title(1, "Route")
            tabs.selected_index = 1 
            


    else:
        with out_map:
          clear_output(wait=True)
          display(initial_map)

def on_button_click_map(btn):
    with out_map:
        clear_output(wait=True)
        display(initial_map)

btn_search.on_click(on_button_click_search)
btn_map.on_click(on_button_click_map)


Output()

In [7]:
# %%cmd
# voila --config=./voila_config.py input_voila.ipynb

In [8]:
if not running_in_voila:
    notebook = "Metro Lyon App.ipynb"
    voila_command = f'voila --config=./voila_config.py "{notebook}"'

    os.system(voila_command)